In [16]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


date = '20210930'


GB = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\01_Corporate_Financing\01_Portfolio\\'+date+'\GB Monthly Loan Outstanding as at 30-Sep-2021 v3.xls', sheet_name='cb os by mkt seg 2') 
#sheet_name='cb os by mkt seg 2'
#sheet_name='cb os by mkt seg'
#sheet_name='SourceCM'
                   
description = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\Python\User defined\description.xlsx", sheet_name='description')


Report_Corp_CRDM = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\11_CRRS\\'+date[:4]+'\\'+date+'\Report_Corp_CRDM_'+date+'.xlsx')



In [21]:
#case to change float to str
GB_try['M_CUS_NO'] = GB_try['M_CUS_NO'].astype('Int64').astype('str')

In [7]:
CRWA = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\12_RDMS\\'+date+'\CRWA Sep 2021 Final (Domestic - All except Retail).xlsx', sheet_name='Context Id=162')


In [8]:
List = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\12_RDMS\\'+date+'\List Master Listing '+date+'-working.xlsx')
#, sheet_name='Sheet2'

In [16]:
GB.columns = GB.columns.str.strip()
GB.columns = GB.columns.str.upper()
GB.columns = GB.columns.str.replace(" ","_")

GB.M_FINANCING_TYPE_CD = GB.M_FINANCING_TYPE_CD.str.strip()
GB.M_FINANCING_TYPE_CD = GB.M_FINANCING_TYPE_CD.str.upper()

#case 20210630 xde facility
#GB.FACILITY = GB.FACILITY.str.strip()
#GB.FACILITY = GB.FACILITY.str.upper()

#case 20211231,20220331 nme M_BNM_BALANCE_SUM_1
GB['M_BNM_BALANCE_SUM']=GB['M_BNM_BALANCE_SUM_1']

GB.M_BNM_BALANCE_SUM.fillna(0, inplace=True)

GB1 = GB.iloc[np.where((GB.M_FINANCING_TYPE_CD=='I')&(GB.FACILITY!='RTO'))] #&(GB.FACILITY!='RTO') case 20210630


In [18]:
#case 20211231
#check M_MSIC_CODE kat excel n let type as general sbb klau x dia jd time value

GB1=GB1[['M_CUS_NO','NAME','M_MSIC_CODE','M_BNM_BALANCE_SUM']]

GB1 = GB1.fillna(0).groupby(['M_CUS_NO','NAME','M_MSIC_CODE'])[['M_BNM_BALANCE_SUM']].sum().reset_index().sort_values(by='M_BNM_BALANCE_SUM', ascending=False)


description.columns = description.columns.str.strip()
description.columns = description.columns.str.replace(" ", "_")
description.columns = description.columns.str.upper()

description.ISC_NOB = description.ISC_NOB.map(str)
description.ISC_NOB = description.ISC_NOB.str.strip()
description.ISC_NOB = description.ISC_NOB.str.upper()

GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.map(str)
GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.str.strip()
GB1.M_MSIC_CODE = GB1.M_MSIC_CODE.str.upper()

join_A = GB1.merge(description[['ISC_NOB','INDUSTRIAL_SECTOR_CODE_(ISC_DESC)2','ISC_PARENT']].drop_duplicates().rename(columns = {'ISC_NOB':'M_MSIC_CODE'}), on='M_MSIC_CODE', how='left').drop('M_BNM_BALANCE_SUM', axis=1)



In [19]:
Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.strip()
Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.upper()
Report_Corp_CRDM.columns = Report_Corp_CRDM.columns.str.replace(" ","_")

Report_Corp_CRDM1 = Report_Corp_CRDM[['GCIF_NUM','RDMS_ADJ_BRR']]

Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.map(str)
Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.str.strip()
Report_Corp_CRDM1.GCIF_NUM = Report_Corp_CRDM1.GCIF_NUM.str.upper()

join_A.M_CUS_NO = join_A.M_CUS_NO.map(str)
join_A.M_CUS_NO = join_A.M_CUS_NO.str.strip()
join_A.M_CUS_NO = join_A.M_CUS_NO.str.upper()

#case 20210630
join_A['M_CUS_NO'] = join_A['M_CUS_NO'].str[:-2]
join_B = join_A.merge(Report_Corp_CRDM1.rename(columns = {'GCIF_NUM':'M_CUS_NO'}).drop_duplicates(), on='M_CUS_NO', how='left')


D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
CRWA.columns = CRWA.columns.str.strip()
CRWA.columns = CRWA.columns.str.upper()
CRWA.columns = CRWA.columns.str.replace(" ","_")

CRWA1 = CRWA[['COUNTERPARTY_ORIGINAL','CUSTOMER_FIELD19','INTERNAL_RATING_ORIGINAL']]

CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.map(str)
CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.str.strip()
CRWA1.COUNTERPARTY_ORIGINAL = CRWA1.COUNTERPARTY_ORIGINAL.str.upper()

join_C = join_B.merge(CRWA1.rename(columns = {'COUNTERPARTY_ORIGINAL':'M_CUS_NO'}).drop_duplicates('M_CUS_NO'), on='M_CUS_NO', how='left').drop_duplicates()


def PSE_SOV(CUSTOMER_FIELD19):
    if CUSTOMER_FIELD19 == 'PSE(SA)':
        return "PSE"
    elif CUSTOMER_FIELD19 == 'SOV(SA)':
        return "SOV"
#    else:
#        return "Unrated"
join_C['PSE/SOV'] = join_C.CUSTOMER_FIELD19.apply(PSE_SOV)

List.columns = List.columns.str.strip()
List.columns = List.columns.str.upper()
List.columns = List.columns.str.replace(" ","_")

List.GCIF_CHILD = List.GCIF_CHILD.map(str)
List.GCIF_CHILD = List.GCIF_CHILD.str.strip()
List.GCIF_CHILD = List.GCIF_CHILD.str.upper()

join_D = join_C.merge(List.rename(columns={"GCIF_CHILD":"M_CUS_NO"}).drop_duplicates(), on='M_CUS_NO', how='left').drop_duplicates()


#join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.apply(lambda x: f'{x:.0f}')
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.astype(str)
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.str.strip()
join_D.GCIF_PARENTS = join_D.GCIF_PARENTS.str.upper()

join_D['GCIF_PARENTS1'] = join_D.GCIF_PARENTS.str[:-2]


join_E = join_D.merge(join_D[['M_CUS_NO','INTERNAL_RATING_ORIGINAL']].rename(columns={'M_CUS_NO':'GCIF_PARENTS1','INTERNAL_RATING_ORIGINAL':'3RD_PARTY'}), on='GCIF_PARENTS1', how='left')


join_E1 = join_E.merge(CRWA1.rename(columns = {'COUNTERPARTY_ORIGINAL':'GCIF_PARENTS1'}).drop_duplicates('GCIF_PARENTS1'), on='GCIF_PARENTS1', how='left',suffixes=('', '_y')).drop_duplicates()

join_E1['CUSTOMER_FIELD19'] = join_E1.CUSTOMER_FIELD19
join_E1.loc[join_E1['CUSTOMER_FIELD19'].isnull(),'CUSTOMER_FIELD19'] = join_E1.CUSTOMER_FIELD19_y


join_E1['3RD_PARTY'] = join_E1['3RD_PARTY']
join_E1.loc[join_E1['3RD_PARTY'].isnull(),'3RD_PARTY'] = join_E1.INTERNAL_RATING_ORIGINAL_y


join_E2 = join_E1.merge(Report_Corp_CRDM1.rename(columns = {'GCIF_NUM':'GCIF_PARENTS1'}).drop_duplicates('GCIF_PARENTS1'), on='GCIF_PARENTS1', how='left',suffixes=('', '_y')).drop_duplicates()

join_E2['3RD_PARTY'] = join_E2['3RD_PARTY']
join_E2.loc[join_E2['3RD_PARTY'].isnull(),'3RD_PARTY'] = join_E2.RDMS_ADJ_BRR_y


join_E2['FINAL_RATING'] = join_E2['INTERNAL_RATING_ORIGINAL']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2.RDMS_ADJ_BRR
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2['3RD_PARTY']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = join_E2['PSE/SOV']
join_E2.loc[join_E2['FINAL_RATING'].isnull(),'FINAL_RATING'] = "Unrated"


join_E2.rename(columns={'M_MSIC_CODE':'BNM_SECTOR_CD',\
                       'INDUSTRIAL_SECTOR_CODE_(ISC_DESC)2':'SECTOR_DESC',\
                      'ISC_PARENT':'SECTOR_PARENT',\
                      'RDMS_ADJ_BRR':'CRRS',\
                      'INTERNAL_RATING_ORIGINAL':'RDMS',\
                      'GCIF_PARENTS1':'3RD_PARTY_PARENT'},inplace=True)

join_E2['SEGMENT'] = '01. CB'

join_E2 = join_E2[['M_CUS_NO',\
                'NAME',\
                'BNM_SECTOR_CD',\
                'SECTOR_DESC',\
                'SECTOR_PARENT',\
                'SEGMENT',\
                'RDMS',\
                'CUSTOMER_FIELD19',\
                'PSE/SOV',\
                'CRRS',\
                '3RD_PARTY_PARENT',\
                '3RD_PARTY',\
                'FINAL_RATING']]

In [23]:
writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\18_Rating_CBBBRSME\BRR_CBBBRSME_'+date[:6]+'(python).xlsx',engine='xlsxwriter')

join_E2.to_excel(writer2, index = False)

writer2.save()

In [ ]:
=='Unrated'  !='N'

In [21]:
join_E2.iloc[np.where(join_E2['FINAL_RATING']=='Unrated')]

,M_CUS_NO,NAME,BNM_SECTOR_CD,SECTOR_DESC,SECTOR_PARENT,SEGMENT,RDMS,CUSTOMER_FIELD19,PSE/SOV,CRRS,3RD_PARTY_PARENT,3RD_PARTY,FINAL_RATING
13,23034947,FGV TRADING SDN BHD,10401A,INTEGRATED PLAYER FOR MANUFACTURE OF CRUDE PALM OIL,03. MANUFACTURING,01. CB,NaN,NaN,None,NaN,N,NaN,Unrated
77,27282547,SHARJAH ELECTRICITY WATER AND GAS AUTHOR,3510XB,ELECTRICITY POWER GENERATION - NATURAL GAS FIRED POWER PLANT,"04. ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY",01. CB,NaN,CORP(SA),None,NaN,N,NaN,Unrated
92,18902204,JCORP CAPITAL EXCELLENCE SDN. BHD.,72105,RESEARCH AND DEVELOPMENT ON AGRICULTURAL SCIENCES,"13. PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES",01. CB,NaN,CORP(SA),None,NaN,N,NaN,Unrated
117,7051644,TNB FUEL SERVICES SDN BHD,72199,OTHER RESEARCH AND EXPERIMENTAL DEVELOPMENT ON NATURAL SCIENCES AND ENGINEERING,"13. PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES",01. CB,NaN,NaN,None,NaN,N,NaN,Unrated
198,21855848,PT FELDA INDO RUBBER,22XXXF,MANUFACTURE OF OTHER RUBBER PRODUCTS N.E.C,03. MANUFACTURING,01. CB,NaN,NaN,None,NaN,N,NaN,Unrated
209,21729247,FGV BIOTECHNOLOGIES SDN. BHD.,3520XA,"MANUFACTURE OF GASEOUS FUELS WITH A SPECIFIED CALORIFIC VALUE, BY PURIFICATION, BLENDING AND OTHER PROCESSES FROM GASES OF VARIOUS TYPES INCLUDING NATURAL GAS","04. ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY",01. CB,NaN,NaN,None,NaN,N,NaN,Unrated
242,26235964,SIME DARBY SWEDISH AUTO SDN. BHD.,451XXB,RETAILER OF NEW PASSENGER MOTOR VEHICLES (EXCLUDE COMMERCIAL VEHICLES),07. WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES,01. CB,NaN,NaN,None,NaN,N,NaN,Unrated
245,24165522,MSM SUGAR REFINERY (JOHOR) SDN BHD,107XX,MANUFACTURE OF OTHER FOOD PRODUCTS,03. MANUFACTURING,01. CB,NaN,NaN,None,NaN,N,NaN,Unrated
332,7768664,AMANAH SAHAM NASIONAL BERHAD,64999,"OTHER FINANCIAL SERVICE ACTIVITIES, EXCEPT INSURANCE/TAKAFUL AND PENSION FUNDING N.E.C.",11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES,01. CB,NaN,CORP(SA),None,NaN,N,NaN,Unrated
352,13291327,ETIQA FAMILY TAKAFUL BERHAD,651XX,INSURANCE/TAKAFUL,11. FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES,01. CB,NaN,CORP(SA),None,NaN,N,NaN,Unrated


# FIX SECTION

In [91]:
Nisa = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\18_Rating_CBBBRSME\BRR_CBBBRSME_202207.xlsx', sheet_name='Sheet1', header=1)

Nisa.columns = Nisa.columns.str.replace(" ","_").str.upper()
Nisa1 = Nisa.iloc[np.where(Nisa.SEGMENT=='01. CB')][['M_CUS_NO','FINAL_RATING']]

In [92]:
Nisa1.shape, join_E.shape

((569, 2), (569, 13))

In [93]:
Syahid1 = join_E[['M_CUS_NO','FINAL_RATING']]

In [94]:
Nisa1.M_CUS_NO = Nisa1.M_CUS_NO.astype(str)
a = Nisa1.merge(Syahid1,on='M_CUS_NO',how='left',indicator=True)

In [95]:
a._merge.value_counts()

both          569
right_only      0
left_only       0
Name: _merge, dtype: int64

In [96]:
a.head(1)

,M_CUS_NO,FINAL_RATING_x,FINAL_RATING_y,_merge
0,23549043,2,2,both


In [97]:
a.FINAL_RATING_x = a.FINAL_RATING_x.astype(str)
a.FINAL_RATING_y = a.FINAL_RATING_y.astype(str)

b = a.iloc[np.where(a.FINAL_RATING_x!=a.FINAL_RATING_y)]

In [68]:
b

,M_CUS_NO,FINAL_RATING_x,FINAL_RATING_y,_merge
31,7051644,4,Unrated,both
189,24165522,11,Unrated,both
233,16892620,10,Unrated,both
257,25804659,10,Unrated,both
402,10042966,10,Unrated,both
418,2532566,10,Unrated,both
439,5151161,10,Unrated,both
463,1484158,10,Unrated,both
464,21516161,11,Unrated,both
471,6015582,11,Unrated,both


In [ ]:
#MASALAH THIRD PARTY SBB AMIK KAT PATCHING JE BUKAN KAT MASTER SKALI

In [73]:
join_E.iloc[np.where(join_E.M_CUS_NO=='24165522')]

,M_CUS_NO,NAME,BNM_SECTOR_CD,SECTOR_DESC,SECTOR_PARENT,SEGMENT,RDMS,CUSTOMER_FIELD19,PSE/SOV,CRRS,3RD_PARTY_PARENT,3RD_PARTY,FINAL_RATING
189,24165522,MSM SUGAR REFINERY (JOHOR) SDN BHD,107XX,MANUFACTURE OF OTHER FOOD PRODUCTS,03. MANUFACTURING,01. CB,NaN,NaN,None,NaN,N,NaN,Unrated


In [75]:
List.GCIF_PARENTS

,GCIF_CHILD,GCIF_PARENTS
0,15413822,14615137
1,15413822,14615137
2,21855848,17199004
3,21955312,11363084
4,3071566,6736153
5,3071566,6736153
6,3925505,17199004
7,7065393,11363084
8,7460731,14615137
9,7460731,14615137


In [ ]:

join_d = pd.concat([join_c.merge(CRWA1.rename(columns={'COUNTERPARTY_ORIGINAL':'GCIF_CHILD'}), on='GCIF_CHILD'),\
                    join_c.merge(CRWA1.rename(columns={'COUNTERPARTY_ORIGINAL':'GCIF_PARENTS'}), on='GCIF_PARENTS')]).drop_duplicates()